# Best CNN-LSTM classifier

This model created from the hyper parameters obtained from hyper parameter tuning with SKlearn.

#### Load dependencies

In [1]:
import os  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, GlobalMaxPooling1D, LSTM
from tensorflow.keras.layers import AveragePooling1D, LeakyReLU , MaxPool1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import roc_auc_score, roc_curve 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import  accuracy_score as score
from tensorflow.python.client import device_lib


# Visiualization imports, Netron and tensorboard:
import netron
from datetime import datetime
from packaging import version
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

import tensorboard
tensorboard.__version__


TensorFlow version:  2.3.1


'2.3.0'

In [2]:
# Try to get reproducable results
from numpy.random import seed
seed(85)
from tensorflow.random import set_seed
set_seed(75)

#### Set hyperparameters

In [3]:
# output directory name:
output_dir = 'output/CNN-LSTM-Final/'
input_dir =  'Z:/Research/dfuller/Walkabilly/studies/smarphone_accel/data/Ethica_Jaeger_Merged/pocket/'
input_file_name = 'pocket-NN-data.npz'

# from the data preparation section we have:
window_size_second = 3
frequency = 30
lenght_of_each_seq = window_size_second * frequency

n_lstm_2 = 128
n_lstm_1 = 128
n_dense_2= 384
n_dense_1= 384
n_conv_3= 512
n_conv_2= 512
n_conv_1= 768
maxpooling_pool_size= 2
k_conv_3= 2
k_conv_2= 2
k_conv_1= 2
dropout_2= 0.2
dropout_1= 0.2
drop_lstm_2 = 0.02
drop_lstm_1 = 0.1
avepooling_pool_size= 2
activation_dense = LeakyReLU()
activation_conv = 'relu'



# training:
epochs = 120
batch_size = 256


#### Load data

For this notebook we use only the **acceleration** data prepared in the DataPrep-Deep notebook.

In [4]:
# import the data
raw_data = np.load(file=input_dir+input_file_name,allow_pickle=True)
accel_array = raw_data['acceleration_data']
meta_array = raw_data['metadata']
labels_array = raw_data['labels']


#### Preprocess data

#### Convert the  labels to integer.
In the raw data format of the labels is String and there are 6 classes. 'Lying', 'Sitting', 'Self Pace walk', 'Running 3 METs',
       'Running 5 METs', 'Running 7 METs' <br>




In [5]:
# change from string to integer so keras.to_categorical can consume it

# could do with factorize method as well
n_class = len(np.unique(labels_array))
class_list, labels_array_int = np.unique(labels_array,return_inverse=True)
print(f'The are {len(class_list)} classes: {class_list}')

The are 6 classes: ['Lying' 'Running 3 METs' 'Running 5 METs' 'Running 7 METs'
 'Self Pace walk' 'Sitting']


#### Convert to categorical values

In [6]:
# labels to categorical

y = to_categorical(labels_array_int, num_classes=n_class)
input_shape = list(accel_array.shape)


#### Design neural network architecture

In [7]:
# The best conv-lstm model!
model = Sequential()
model.add(Conv1D(n_conv_1, k_conv_1, activation=activation_conv, input_shape=input_shape[1:]))
model.add(MaxPool1D(pool_size = maxpooling_pool_size))
model.add(Conv1D(n_conv_2, k_conv_2, activation=activation_conv))
model.add(AveragePooling1D(pool_size = avepooling_pool_size))
model.add(Conv1D(n_conv_3, k_conv_3, activation=activation_conv))

# LSTM layers
model.add(LSTM(n_lstm_1, dropout=drop_lstm_1, return_sequences=True)) 
model.add(LSTM(n_lstm_2, dropout=drop_lstm_2, return_sequences=True))

# Dense layers
model.add(Flatten())
model.add(Dense(n_dense_1, activation=activation_dense))
model.add(Dropout(dropout_1))
model.add(Dense(n_dense_2, activation=activation_dense))
model.add(Dropout(dropout_2))
model.add(Dense(n_class, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 89, 768)           5376      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 44, 768)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 43, 512)           786944    
_________________________________________________________________
average_pooling1d (AveragePo (None, 21, 512)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 512)           524800    
_________________________________________________________________
lstm (LSTM)                  (None, 20, 128)           328192    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           1

### Splitting and shuffeling the data

In [9]:
# split and shuffle
x_train, x_valid, y_train, y_valid = train_test_split(
     accel_array, y, test_size=0.1, random_state=65)

#### Prepare to save each epoch
The weights and a log for tensorboard are saved. Later we load back the best model

In [10]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+
                                  "/weights.{epoch:02d}.hdf5")

# Define the Keras TensorBoard callback.
logdir=  "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


#### Create a folder to save each epoch of training

In [11]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [12]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((58278, 90, 3), (58278, 6), (6476, 90, 3), (6476, 6))

#### Train!

In [ ]:
model.fit(x_train, y_train, 
         batch_size=batch_size, epochs=epochs, verbose=1, 
         validation_data=(x_valid, y_valid), 
         callbacks=[modelcheckpoint, tensorboard_callback])

Epoch 1/120
  1/228 [..............................] - ETA: 0s - loss: 1.7924 - accuracy: 0.1914WARNING:tensorflow:From C:\Users\dfuller\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
228/228 [==============================] - 18s 79ms/step - loss: 0.9398 - accuracy: 0.6033 - val_loss: 0.6706 - val_accuracy: 0.7353
Epoch 2/120
228/228 [==============================] - 17s 74ms/step - loss: 0.5654 - accuracy: 0.7771 - val_loss: 0.4588 - val_accuracy: 0.8173
Epoch 3/120
228/228 [==============================] - 16s 72ms/step - loss: 0.4089 - accuracy: 0.8427 - val_loss: 0.3703 - val_accuracy: 0.8544
Epoch 4/120
228/228 [==============================] - 17s 73ms/step - loss: 0.3639 - accuracy: 0.8586 - val_loss: 0.3382 - val_accuracy: 0.8698
Epoch 5/120
228/

#### Load the best epoch's weight

In [ ]:
model.load_weights(output_dir+"/weights.110.hdf5") 

## Evaluate

Make predictions and check the results

In [ ]:
y_hat = model.predict(x_valid)

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

#### Histogram of probabilities for each class
If most probabilities are around either zero or one, it means the model was confidant in classification

In [ ]:
df = pd.DataFrame(y_hat, columns=class_list)
_ = df.hist(figsize=(15,10), color='brown')

In [ ]:
# Calculate ROC and Accuracy
pct_auc = roc_auc_score(y_valid, y_hat)*100.0
y_pred_classes = y_hat.round()
acc = score(y_valid, y_pred_classes) * 100

In [ ]:
print("The model accuracy is: {:0.2f}".format(acc))
print("The area under ROC curve is: {:0.2f}".format(pct_auc))

In [ ]:
# Convert the predicted outcomes to categorical variable to compare with validation data, just for checking

float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0:7].round(3))

In [ ]:
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])

In [ ]:
ydf.head()

In [ ]:
# convert probability to class
y_pred_class = np.argmax(y_hat, axis=1)
y_class = np.argmax(y_valid, axis = 1)

In [ ]:
sns.heatmap(cf_matrix, annot=True, fmt = '1', xticklabels=class_list, yticklabels=class_list, cmap='ocean')
# cf_matrix

### plot confusion matrix
First use seaborn, then a custom stript.

In [ ]:
# Create confucion matrix:
cf_matrix = confusion_matrix(y_class, y_pred_class, labels=range(len(class_list)))


# the classes are not in order, for example 'sitting' is not following 'lying'. Sort them:

confusion_matrix_df = pd.DataFrame(cf_matrix, index=class_list, columns=class_list)
confusion_matrix_df = confusion_matrix_df[['Lying','Sitting', 'Self Pace walk',\
                                           'Running 3 METs','Running 5 METs', 'Running 7 METs']]
confusion_matrix_df = confusion_matrix_df.reindex(['Lying', 'Sitting', 'Self Pace walk',\
                                                   'Running 3 METs','Running 5 METs', 'Running 7 METs'])

In [ ]:
plt.figure(figsize = (13,10))

_ = sns.heatmap(confusion_matrix_df, annot=True, fmt = '1', cmap='rocket_r') # try cmap='rocket', 'PuBuGn', 'magma'


In [ ]:
import prettyconfusion as pf
pf.pretty_plot_confusion_matrix(confusion_matrix_df,cmap='PuBuGn', cbar='oranges'
                                ,fz = 13, lw=1, figsize=[12,12])

## Visualize performance metrics and the model's structure

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs if you don't want to see it in tensorflow

In [ ]:
%tensorboard --logdir logs

In [ ]:
# All stored weights have the same structure. Load one of them to netron and save the image 
netron.start(file=output_dir+"/weights.110.hdf5")